# Machine Learning Engineer Nanodegree
## Capstone Proposal
Henry Maguire 
3rd October 2017

## Proposal
I would like to make an end-to-end machine translation system that is capable of translating phrases from French to English, to a higher degree of accuracy than a benchmark dictionary based system.

### Domain Background
<!--- - provide brief details on the background information of the domain from which the project is proposed. 
#- Historical information relevant to the project should be included. 
#- It should be clear how or why a problem in the domain can or should be solved.
#- Related academic research should be appropriately cited in this section, including why that research is relevant. 
#- A discussion of your personal motivation for investigating a particular problem in the domain is encouraged but not #required.
#_(approx. 1-2 paragraphs)_ -->

Over the years of development in machine learning, performance in many tasks has improved substantially. A clear example is speech recognition, where Google's speech recognition system is now "95% accurate" in the English language and has "improved 20%" since 2013 alone. In many areas the field of NLP still lags behind, with chatbots still being restricted to very closed domain conversations and remain very unconvincing when passing for humans, despite many recent technological improvements. However, the application of machine learning in language translation has been very successful, with the Google neural MT system [reducing errors](https://research.google.com/pubs/pub45610.html) by 60% compared with phrase-based systems. Statistical and rule-based machine translation has been used since World War Two, which require a large amount of domain knowledge and human input, hard-coding of grammar into large binary trees, etc. Many developments have been made in using Bayesian inference, [maximum likelihood](http://aclweb.org/anthology/H94-1100) of translations, [log-linear models](https://homes.cs.washington.edu/~nasmith/papers/smith.tut04.pdf) and other sophisticated SMT techniques - the accuracy of these systems has slowly improved over time with computational power. Other approaches have involved translating both source and target phrases into a common intermediate language, again using pre-developed knowledge of the structure of each language.

Cutting edge solutions often use a [combination of established grammatical models and deep neural networks](https://arxiv.org/pdf/1406.1078.pdf), since deep neural networks are able to approximate arbitrary functions, which makes them useful in supervised learning applications. The backpropagation algorithm allows correlations to be learned from data by updating the weights of a neural network iteratively, based on how far from the ideal scenario an approximation/prediction was. Neural networks can also take advantage of the symmetry of a problem. [Recurrent Neural Network](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) share parameters across "time steps" - so that each time we see the word "cat" we don't have to re-learn the weights in the network. This also allows sequential data to be handled, with a varying number of inputs mapping to a fixed number of outputs, so it generalises very well to natural language processing tasks. This means that all you need to develop a translation system is a load of parallel text in both the source and target language, as well as enough computing power. It turns out that both requirements have recently become a lot easier to satisfy, with free access to large parallel corpora and relatively cheap GPUs.

The [Google MT system](https://arxiv.org/pdf/1609.08144.pdf) deploys two Deep Recurrent Neural Networks which are connected end-to-end, one which generates fixed size vector representations for variable length input phrases in the source language (the encoder) and another which takes these vectors and generates variable length sequence representations of them in the target language (the decoder). This is known as [Encoder-Decoder architecture](https://arxiv.org/pdf/1406.1078.pdf). The final layer then projects a probability distribution over predicted words in the target language at each time step. Simple RNN cells suffer from a number of pathologies, such as [vanishing gradients](http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/), which make it difficult for them to keep track of long-term language dependencies. To get around this the Google NMT system uses Long Short-Term cells, which are gated units that can adaptively learn to forget and remember different weights depending on the context of a current word - this theoretically allows long-term dependencies to be learned. The effectiveness of the system increases greatly with the depth of the network so they also use 8 decoder and 8 encoder LTSM layers, which in turn expands the hardware requirements substantially. Another breakthrough of the Google system is in the use of [attention mechanisms](https://arxiv.org/pdf/1409.0473.pdf), whereby the entire output of the encoder layer is made visible to the (first) decoder layer throughout the computation. This allows the decoder to keep track of the entire context of the word at all timesteps, using important summary representations of the whole phrase. Another advanced technique is in the handling of rare, where they split up individual words into sub units, for example `"going"` becomes `"go", "ing"` etc - this would require some kind of auxillary learning algorithm (I probably won't do this). 





### Problem Statement

<!--- - Clearly describe the problem that is to be solved. 
- The problem described should be well defined and should have at least one relevant potential solution. 
- Additionally, describe the problem thoroughly such that it is clear that the problem is **quantifiable** (the problem can be expressed in mathematical or logical terms) , **measurable** (the problem can be measured by some metric and clearly observed), and **replicable** (the problem can be reproduced and occurs more than once). -->

The problem of machine translation requires decoding the *meaning* of phrases and then encoding this meaning into an entirely different representation/script. The meaning of phrases can be embedded in a distributed vector representation, much like the meaning of words can be with methods like [word2vec](), as a kind of conditional probability distribution. In theory, RNNs can learn the types of conditional probability distributions which are necessary for this task e.g.

    Given the phrase "Mr Smith was walking" what is the probability
    of each word in the vocabulary appearing next?

Effectively, variable length sequences can be mapped to fixed length vectors, enabling sequences of words to be generated probabilistically either one word at a time or via beam search etc (such as in [Andrej Karpathy's blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) which performs this task at the character level to generate almost compilable Linux code)! The probability distributions over target words increases with vocabilary size, so the vectors can become very sparse and the number of model parameters often increases exponentially - this is another huge problem in machine translation and is a reason why closed domain MT systems (say, political, military or legal documents) have long outperformed those used in open domains (arbitrary conversation). 

Translation also cannot happen on a word-by-word basis, due to idiomatic phrases and long term grammatical dependencies, for example:

    __*Mr*__ Smith was walking to the shop when a bird flew into __*his*__ head.

Here the words "Mr" and "his" share the same gender, but this dependency is *hidden* for 11 words, this makes it very difficult for many types of machine translation systems. Simple RNNs, whose activations are always less than one due to the output range of the non-linearities applied (tanh, sigmoid, etc), lead to very small gradients over just a few time steps as the activations diminish further and further when multiplied by weights. This means that although not theoretically limited in their memory, simple RNNs cannot learn these dependencies in practice and so other architectures (which I will go into) or regularisation techniques (say imposing a threshold on the norm of the gradient) need to be used. 

Another enormous problem is that even if we have a fixed vector representation of a phrase or word in our source language, this vector may live in a completely different vector space to the target phrase and so we have no way to measure similarity directly. This is where the magic of encoder-decoder architecture comes in, which I will discuss below.

### Datasets and Inputs
<!--- - The dataset(s) and/or input(s) being considered for the project should be thoroughly described, such as how they relate to the problem and why they should be used.
- Information such as how the dataset or input is (was) obtained, and the characteristics of the dataset or input, should be included with relevant references and citations as necessary
- It should be clear how the dataset(s) or input(s) will be used in the project and whether their use is appropriate given the context of the problem. -->

TO train the neural networks we need a corpus of text in both the target language and the source language - for this I have decided to use the [WMT'14](https://github.com/bicici/ParFDAWMT14) English-French parallel corpora, which are transcribed TED talks. Other datasets are available, however these tend to be closed domain (Canadian political documents) or very casual language (tweets). Since I don't speak French, I will use French and English as the input/source and output/target languages respectively, so that I can evaluate the generated translations qualitatively myself as well as via any performance metrics that I define below, this just aids in the debugging process. In order to process the information easily and to reduce the possiblity of alignment errors, the two corpora need to be broken up into parallel lists of phrases. These prhrases then need to be split up into tokens (words, punctuation and abbreviations), so that each one can be input into the model at each time step.

To reduce the computational requirements I will reduce the vocabulary to cover about 80-85% of the data, I will replace out of vocabulary words with a `<UNK>` tag. Due to the long-tailed [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4176592/](Zipf's law distribution) of word frequencies, this number of words in the vocabulary will be perhaps orders of magnitude smaller than requiring 90-95% of the data be captured by the vocabulary.

## Solution Statement

<!--- - clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. 
- Additionally, describe the solution thoroughly such that it is clear that the solution is quantifiable (the solution can be expressed in mathematical or logical terms) , measurable (the solution can be measured by some metric and clearly observed), and replicable (the solution can be reproduced and occurs more than once). -->

As outlined above, we need a way of mapping from variable length input sequences (in the source language) to variable length output sequences (target language), which can be done using [sequence-to-sequence models](https://arxiv.org/pdf/1409.3215.pdf). 
![seq2seq architecture](pictures/2-seq2seq-feed-previous.png)
Essentially, a phrase in the target language is fed into a recurrent neural network one word at a time which gets encoded in the final hidden RNN layer. The final encoder layer is then passed to the decoder layer which generates a representation of the phrase in the target language. During training the entire network is unrolled across some truncated number of timesteps and the target phrase is fed in as both the input sequence and the "gold truth" training example. The output of the decoder layer is converted to a probability.
Build the end-to-end translation system, perhaps comparing:
- Vanilla RNN units, understanding pathologies
- LSTMs
- Bi-directional LSTMs
- Reverse source sentences

Two other problems are how to quantify how close our prediction and the actual result are and how to make the output intelligible/appropriate. These form the basis of bleeding edge NLP/machine translation research.

I will make my implementation in Tensorflow, since this has a dedicated sequence-to-sequence module and it is the Deep Learning library I have most experience with.

### Benchmark Model

A Benchmark Model could be to just translate word for word the source sentence using dictionary definitions, via some [online translation software API](https://pypi.python.org/pypi/googletrans). This may be cheating since we would have to reference some third party dictionary definitions, but it would provide a simple baseline with which we can compare. This method would no doubt miss all of the idiomatic phrases which may appear in the original corpus and is likely to get the ordering of adjectives and gendered articles confused. An example of a likely benchmark error is that the phrase "the black cat" in English becomes "le chat noir" - which uses the masculine definite pronoun (which doesn't exist in English) and has the ordering of "chat" and "noir" reversed from the source phrase. 

### Evaluation Metrics

There are two points at which the accuracy of the model will be assessed. The first is during training when the decoder layer predicts the next word in the sequence for each timestep - for this I will use some standard loss such as cross-entropy (with softmax) to find the score (probability) vector.
The second is during testing, to see the efficacy of the translation machine. For this I could use the [BLEU metric](https://en.wikipedia.org/wiki/BLEU) to measure the accuracy of both the word-for-word translation and the Deep Learning approach. The BLEU metric is the most popular used in the field of machine translation since it is known to [correlate well with human judgment](https://www.aclweb.org/anthology/E/E06/E06-1032.pdf).

The [BLEU scoring method](https://en.wikipedia.org/wiki/BLEU) uses n-gram frequency in the machine output and human translated phrase, whereby each sequence of words up to length `n` in the target and output translation are recorded. The BLEU score is based on the ratio of: the number of n-grams in the predicted phrase which appear in the target phrase, `m` and the total number of words in the predicted phrase, `w_t`. A score of `P=m/w_t=1` is a perfect score. This standalone metric would value translations which are dominated by just a single n-gram which is found in the target phrase. The value in the numerator is therefore augmented such that the number of times an n-gram in the predicted phrase can be counted is limited to the number of times it appears in the target phrase. For example, using a 2-gram BLEU score:

Predicted phrase: **The dog the dog the dog**              

Target phrase: **The dog sat on the log**


In the predicted phrase there are five 2-grams: **the dog** appears three times and **dog the** appears twice, of these only **the dog** appears in the translation and it appears once. Without the augmentation, the 2-gram score would be `P=3/5` but with the augmentation the score would be `P=1/5`. Unfortunately, `n` is a hyperparameter, but fortunately research has shown that `n=4` seems to agree most strongly with "monolingual human judgements". Also, small values of `n` seem to measure how much information was retained in the translation whereas larger values are a measure of the readability/fluency. Given this latter definition, I expect that the benchmark model outlined above may score a reasonable unigram BLEU score but will probably not score very highly with respect to larger N-grams.



## Project Design
<!---(approx. 1 page)

- Summarize a theoretical workflow for approaching a solution given the problem.
- Provide thorough discussion for what strategies you may consider employing, what analysis of the data might be required before being used, or which algorithms will be considered for your implementation.
- The workflow and discussion that you provide should align with the qualities of the previous sections.
- Additionally, you are encouraged to include small visualizations, pseudocode, or diagrams to aid in describing the project design, but it is not required.
- The discussion should clearly outline your intended workflow of the capstone project. -->

The overall structure of the project will be:

- Load English and French datasets into memory<br>
- Preprocess the data by tokenizing, removing capitalisation, limiting vocabulary size and replacing out of vocabulary words with the `<UNK>` token.<br>
- Write the code for the benchmark dictionary based model, utilising the Python [GoogleTrans package](https://pypi.python.org/pypi/googletrans) as a dictionary. Some pseudocode might be 

`for phrase in corpus:` <br/>
&nbsp;&nbsp;&nbsp;&nbsp; `translated_phrase = []`

&nbsp;&nbsp;&nbsp;&nbsp; `for word in phrase:`

&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; `if word == '<UNK>':`

&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; `translated_phrase.append('<UNK>')`

&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; `elif word == '<EOS>'`

&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; `translated_phrase.append('<EOS>')`

&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; `else:`

&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;`translated_phrase.append(translate(word, 'fr'))`

- Find [word embeddings](https://spinningbytes.com/resources/embeddings/) which have been pre-trained on 200 million (ENG) and 300 million (FR) tweets, with an embedding size of 52. A package called `gensim` can be used to easily load and manipulate word-to-vec embeddings and the [API documentation](http://radimrehurek.com/gensim/models/word2vec.html) looks fairly straightforward to use.
- Load in pretrained word embeddings and find embeddings for each vocabulary word, if there is no pretrained embedding available just initialise to Gaussian random distribution. 
- Implement BLEU metric either by hand or using the [NLTK implementation](http://www.nltk.org/_modules/nltk/translate/bleu_score.html)
- Build the single-layer RNN-RNN translation system
    - build a batching system to split up the data and pad with zeros, choose whether time major (vertical) or not.<br>
    - Make tensorflow placeholders for the encoder inputs, decoder targets and decoder inputs <br>
    - Convert the input integer sequences to sequences of embedding vectors (equivalent to one hot encoding each word as a vector and multiplying it by the embedding matrix?)<br>
    - Define the encoder and decoder cells (either LSTM or RNN), requires specifying the number of hidden LSTM units for each (enc and dec must be equal).<br>
    - To handle variable length sequences we use tf.nn.dynamic_rnn for both encoder and decoder, which uses while loops to generate graphs with the appropriate number of timesteps (max length in batch) at run time. The inputs to the encoder (decoder) are the embedded phrase batches and embedded decoder inputs (these will vary between training and testing), respectively. We also set the initial state of the first hidden unit in the decoder to be the final state of the final encoder layer (this is the Encoder-Decoder magic).<br>
    - Next we calculate the projection layer, which is just a linear tranformation which converts the dimension of the decoder output to be the same as the target vocabulary size, to enable scores to be calculated. Due to batching, the output will be a tensor with dimension `(decoder max step number, decoder batch size, target vocab size)`, so we may need to flatten this to apply a linear layer.<br>
    - Take the decoder outputs and perform an argmax over the scores for the different next words. i.e. maximizing the score of the predicted word. Also then apply a softmax function to the logits/scores to get probabilities.<br>
    - Calculate the cross-entropy between the target and predicted word. It may be necessary to one-hot encode the target words, to easily compare the predicted and target words.<br>
    - At this point we will have a tensor of dimension `(decoder max step number, decoder batch size)` since we will have calculated how well our prediction has done for each next-word prediction in the batch. We can calculate a collective measure of batch loss by taking the mean of all of the cross-entropies. <br>
    - I will perform backpropagation over the loss, which can be done using the simple Tensorflow API by defining an optimiser (say SGD, ADAGrad or Adam) and then calling an minimise method like
    `tf.train.AdamOptimizer().minimize(loss)`.
    I'm not certain about how this works on the Tensorflow graph but I will find out when it comes to implementing it.

![seq2seq at training time](pictures/1-seq2seq.png)
- Now the model should be ready to train. I will feed the source phrases into the encoder, one word at a time, until the `<EOS>` tag is reached. Then the target phrases will be fed in word by word into the decoder layer.<br>
- Test trained model against the training set. Evaluate BLEU metric.<br>
- Compare to the benchmark model.<br>

### Computational resources

My pesonal computer has 16GB of RAM and 4 cores, so will be adequate for development. For training, computational resources may become an issue although I have free access to 2-64 cores. If more computing power is needed then perhaps I could use a GPU on [Floyd](https://www.floydhub.com/) or [AWS](https://aws.amazon.com/).

<!--- *In this final section, summarize a theoretical workflow for approaching a solution given the problem. Provide thorough discussion for what strategies you may consider employing, what analysis of the data might be required before being used, or which algorithms will be considered for your implementation. The workflow and discussion that you provide should align with the qualities of the previous sections. Additionally, you are encouraged to include small visualizations, pseudocode, or diagrams to aid in describing the project design, but it is not required. The discussion should clearly outline your intended workflow of the capstone project.* -->

-----------

<!--- **Before submitting your proposal, ask yourself. . .**

- Does the proposal you have written follow a well-organized structure similar to that of the project template?
- Is each section (particularly **Solution Statement** and **Project Design**) written in a clear, concise and specific fashion? Are there any ambiguous terms or phrases that need clarification?
- Would the intended audience of your project be able to understand your proposal?
- Have you properly proofread your proposal to assure there are minimal grammatical and spelling mistakes?
- Are all the resources used for this project correctly cited and referenced? -->